```@meta
CurrentModule = CounterfactualExplanations 
```


In [ ]:
#| echo: false
include("$(pwd())/docs/setup_docs.jl")
eval(setup_docs)

# `REVISEGenerator`

REVISE is a Latent Space generator introduced by @joshi2019realistic.

## Description

The current consensus in the literature is that Counterfactual Explanations should be realistic: the generated counterfactuals should look like they were generated by the data-generating process (DGP) that governs the problem at hand. With respect to Algorithmic Recourse, it is certainly true that counterfactuals should be realistic in order to be actionable for individuals.^[In general, we believe that there may be a trade-off between creating counterfactuals that respect the DGP vs. counterfactuals reflect the behaviour of the black-model in question - both accurately and complete.] To address this need, researchers have come up with various approaches in recent years. Among the most popular approaches is **Latent Space Search**, which was first proposed in @joshi2019realistic: instead of traversing the feature space directly, this approach relies on a separate generative model that learns a latent space representation of the DGP. Assuming the generative model is well-specified, access to the learned latent embeddings of the data comes with two **advantages**:

1. Since the learned DGP is encoded in the latent space, the generated counterfactuals will respect the learned representation of the data. In practice, this means that counterfactuals will be realistic.
2. The latent space is typically a compressed (i.e. lower dimensional) version of the feature space. This makes the counterfactual search less costly.

There are also certain **disadvantages** though:

1. Learning generative models is (typically) an expensive task, which may well outweigh the benefits associated with utlimately traversing a lower dimensional space. 
2. If the generative model is poorly specified, this will affect the quality of the counterfactuals.^[We believe that there is another potentially crucial disadvantage of relying on a separate generative model: it reallocates the task of learning realistic explanations for the data from the black-box model to the generative model.]

Anyway, traversing latent embeddings is a powerful idea that may be very useful depending on the specific context. This tutorial introduces the concept and how it is implemented in this package. 

## Usage

The approach can be used in our package as follows:


In [ ]:
#| output: true
generator = REVISEGenerator()
ce = generate_counterfactual(x, target, counterfactual_data, M, generator)
plot(ce)

### Worked 2D Examples

Below we load 2D data and train a VAE on it and plot the original samples against their reconstructions.


In [ ]:
# output: true

counterfactual_data = load_overlapping()
X = counterfactual_data.X
y = counterfactual_data.y
input_dim = size(X, 1)
using CounterfactualExplanations.GenerativeModels: VAE, train!, reconstruct
vae = VAE(input_dim; nll=Flux.Losses.mse, epochs=100, λ=0.01, latent_dim=2, hidden_dim=32)
flux_training_params.verbose = true
train!(vae, X, y)
X̂ = reconstruct(vae, X)[1]
p0 = scatter(X[1, :], X[2, :], color=:blue, label="Original", xlab="x₁", ylab="x₂")
scatter!(X̂[1, :], X̂[2, :], color=:orange, label="Reconstructed", xlab="x₁", ylab="x₂")
p1 = scatter(X[1, :], X̂[1, :], color=:purple, label="", xlab="x₁", ylab="x̂₁")
p2 = scatter(X[2, :], X̂[2, :], color=:purple, label="", xlab="x₂", ylab="x̂₂")
plt2 = plot(p1,p2, layout=(1,2), size=(800, 400))
plot(p0, plt2, layout=(2,1), size=(800, 600))

Next, we train a simple MLP for the classification task. Then we determine a target and factual class for our counterfactual search and select a random factual instance to explain.


In [ ]:
M = fit_model(counterfactual_data, :MLP)
target = 2
factual = 1
chosen = rand(findall(predict_label(M, counterfactual_data) .== factual))
x = select_factual(counterfactual_data, chosen)

Finally, we generate and visualize the generated counterfactual:


In [ ]:
#| output: true

# Search:
generator = REVISEGenerator()
ce = generate_counterfactual(x, target, counterfactual_data, M, generator)
plot(ce)

### 3D Example

To illustrate the notion of Latent Space search, let's look at an example involving 3-dimensional input data, which we can still visualize. The code chunk below loads the data and implements the counterfactual search. 


In [ ]:
# Data and Classifier:
counterfactual_data = load_blobs(k=3)
X = counterfactual_data.X
ys = counterfactual_data.output_encoder.labels.refs
M = fit_model(counterfactual_data, :MLP)

# Randomly selected factual:
x = select_factual(counterfactual_data,rand(1:size(counterfactual_data.X,2)))
y = predict_label(M, counterfactual_data, x)[1]
target = counterfactual_data.y_levels[counterfactual_data.y_levels .!= y][1]

# Generate recourse:
ce = generate_counterfactual(x, target, counterfactual_data, M, generator)

The figure below demonstrates the idea of searching counterfactuals in a lower-dimensional latent space: on the left, we can see the counterfactual search in the 3-dimensional feature space, while on the right we can see the corresponding search in the latent space.


In [ ]:
#| echo: false
#| output: true

# Feature:
p1 = scatter3d(X[1,:],X[2,:],X[3,:],group=vec(Int.(ys)), title="Feature space (3d)", palette=:rainbow)
X_path = reduce(hcat,path(ce))
scatter3d!([X_path[1,end]],[X_path[2,end]],[X_path[3,end]],colour=:yellow,ms=10,label="Counterfactual")
scatter3d!(X_path[1,1:(end-1)],X_path[2,1:(end-1)],X_path[3,1:(end-1)],colour=:yellow,ms=5,label="")

# Latent:
p2 = plot(ce.data.generative_model, X, ys; title="Latent space (2d)")
X_path = reduce(hcat,path(ce, feature_space=false))
scatter!([X_path[1,end]],[X_path[2,end]],colour=:yellow,ms=10,label="Counterfactual")
scatter!([X_path[1,1:(end-1)]],[X_path[2,1:(end-1)]],colour=:yellow,ms=5,label="")

plot(p1,p2, legend=false, size=(800,350))

## MNIST data

Let's carry the ideas introduced above over to a more complex example. The code below loads MNIST data as well as a pre-trained classifier and generative model for the data.


In [ ]:
using CounterfactualExplanations.Models: load_mnist_mlp, load_mnist_ensemble, load_mnist_vae
counterfactual_data = load_mnist()
X, y = CounterfactualExplanations.DataPreprocessing.unpack_data(counterfactual_data)
input_dim, n_obs = size(counterfactual_data.X)
M = load_mnist_mlp()
vae = load_mnist_vae()

In [ ]:
#| echo: false
#| output: true

test_data = load_mnist_test()
Markdown.parse(
  """
  The F1-score of our pre-trained image classifier on test data is: $(round(model_evaluation(M, test_data)[1], digits=2))
  """
)

Before continuing, we supply the pre-trained generative model to our data container:


In [ ]:
counterfactual_data.generative_model = vae # assign generative model

Now let's define a factual and target label:


In [ ]:
# Randomly selected factual:
Random.seed!(2023)
factual_label = 8
x = reshape(X[:,rand(findall(predict_label(M, counterfactual_data).==factual_label))],input_dim,1)
target = 3
factual = predict_label(M, counterfactual_data, x)[1]

In [ ]:
#| echo: false
#| output: true

Markdown.parse(
  """
  Using REVISE, we are going to turn a randomly drawn $(factual_label) into a $(target).
  """
)

The API call is the same as always:


In [ ]:
γ = 0.95
# Define generator:
generator = REVISEGenerator(opt=Flux.Adam(0.5))
# Generate recourse:
ce = generate_counterfactual(x, target, counterfactual_data, M, generator; decision_threshold=γ)

The chart below shows the results:


In [ ]:
#| echo: false
#| output: true

image_size = 300
p1 = plot(
    convert2image(MNIST, reshape(x,28,28)),
    axis=nothing, 
    size=(image_size, image_size),
    title="Factual: $(factual_label)"
)
savefig(p1, joinpath(www_path, "mnist_factual.png"))
target_prob = round(target_probs(ce)[1],digits=2)
ŷ = CounterfactualExplanations.counterfactual_label(ce)[1]
p2 = plot(
    convert2image(MNIST, reshape(CounterfactualExplanations.counterfactual(ce),28,28)),
    axis=nothing, 
    size=(image_size, image_size),
    title="Counterfactual: $(ŷ) (p=$(target_prob))"
)
plot(p1,p2;size=(image_size * 2,image_size), layout=(1,2))

## References